In [1]:
import tensorflow as tf
import numpy as np
import os,gc,re

D:\Anaconda\Initial\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.Session()

In [3]:
def load_word2vec_1(emb_path,old_weights):
    #这个是将随机赋值的字典换成新的字典，字不变，但是对应的vec改成了preTrain的word2vec
    new_weights = old_weights
    
    assert os.path.isfile(emb_path)
    print('Loading pretrained embeddings from {}...'.format(emb_path))
    
    pre_trained = {}
    with open(emb_path,"r+",encoding="utf-8") as file:
        for line in file:
            content = line.strip().split()
            if len(content)>2:
                pre_trained[content[0]] = np.array([float(each) for each in content[1:]],dtype=np.float32)
                
    for word in new_weights.keys():
        remove_special_characters = re.sub("[^\w]","",word) #把非数字字母下划线 中文(\w能识别中文) 的特殊字符抹去
        if word in pre_trained:
            vec = pre_trained[word] 
        elif remove_special_characters in pre_trained:
            vec = pre_trained[remove_special_characters] 
        elif word.lower() in pre_trained:
            vec = pre_trained[word.lower()]
        elif re.sub('\d', '0', word) in pre_trained:
            vec = pre_trained[re.sub('\d', '0', word)]
        else:
            vec = np.array(new_weights[word],dtype=np.float32)
        new_weights[word] = vec
    
    pre_trained=dict()
    gc.collect()
    
    return new_weights

def load_word2vec_2(sess,emb_path,old_weights,to_be_assigned): #这里的old_weights必须是已经排好序的了
    #这里是直接将随机初始化的总表直接修改，修改完后直接传回给随机初始化的总表
    #要注意，从总表中拿数据可以紧接着总表随机初始化之后，但是sess.run()的从总表中拿数据，则需要在这个函数之后
    words_num = to_be_assigned.shape[0].value
    words_dim = to_be_assigned.shape[1].value
    init_emb = np.random.uniform(-1.0,1.0,(words_num,words_dim))
    
    new_weights = old_weights
    
    assert os.path.isfile(emb_path)
    print('Loading pretrained embeddings from {}...'.format(emb_path))
    
    pre_trained = {}
    with open(emb_path,"r+",encoding="utf-8") as file:
        for line in file:
            content = line.strip().split()
            if len(content)>2:
                pre_trained[content[0]] = np.array([float(each) for each in content[1:]],dtype=np.float32)
    
    for idx,word in enumerate(old_weights.keys()):
        remove_special_characters = re.sub("[^\w]","",word) #把非数字字母下划线 中文(\w能识别中文) 的特殊字符抹去
        if word in pre_trained:
            value = pre_trained[word]
        elif remove_special_characters in pre_trained:
            value = pre_trained[remove_special_characters]
        elif word.lower() in pre_trained:
            value = pre_trained[word.lower()]
        elif re.sub('\d', '0', word) in pre_trained:
            value = pre_trained[re.sub('\d', '0', word)]
        else:
            value = old_weights[word]
        init_emb[idx] = np.asarray(value).astype(np.float32)
    
    pre_trained=dict()
    gc.collect()
    
    sess.run(to_be_assigned.assign(init_emb))
    

In [4]:
# test for load_word2vec_1
emb_path = "./emb.txt"
old_weights = {"a":66,"d":77,"g":88,"i":99}
new_weight = load_word2vec_1(emb_path,old_weights)
print(new_weight)

Loading pretrained embeddings from ./emb.txt...
{'a': array([1., 2., 3., 4., 5.], dtype=float32), 'd': array([4., 2., 3., 4., 5.], dtype=float32), 'g': array(88., dtype=float32), 'i': array(99., dtype=float32)}


In [5]:
# test for load_word2vec_2
emb_path = "./emb.txt"
to_be_assigned = tf.Variable(dtype=tf.float32,name="origin",initial_value=np.arange(20).reshape(4,5))
sess.run(tf.global_variables_initializer())  #因为又定义了一个tf.Variable,所以这里需要再初始化一下
print(sess.run(to_be_assigned),"\n")   #未被修改之前
value = np.array([8,8,8,8,8],dtype=np.float32)
old_weights = {"a":value,"b":value,"g":value,"h":value}
print("old_weights:\n",old_weights,"\n")
load_word2vec_2(sess,emb_path,old_weights,to_be_assigned)
print(sess.run(to_be_assigned))   #被修改之后

[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]] 

old_weights:
 {'a': array([8., 8., 8., 8., 8.], dtype=float32), 'b': array([8., 8., 8., 8., 8.], dtype=float32), 'g': array([8., 8., 8., 8., 8.], dtype=float32), 'h': array([8., 8., 8., 8., 8.], dtype=float32)} 

Loading pretrained embeddings from ./emb.txt...
[[1. 2. 3. 4. 5.]
 [2. 2. 3. 4. 5.]
 [8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8.]]
